## Installing libraries

In [19]:
# Looking available GPU
!nvidia-smi -L

In [20]:
# Istalling efficientnet 
# Now models can be build with Keras or Tensorflow frameworks 
!pip install -q efficientnet

In [21]:
# Install custom image data generator for tensorflow.keras that supports albumentations
!pip install git+https://github.com/mjkvaak/ImageDataAugmentor

In [22]:
# Install Python library for image augmentation with large set of image transformations
!pip install -U albumentations==1.0.3

In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import zipfile
import csv
import sys
import os
import itertools
from skimage import io

import tensorflow as tf
from tensorflow.keras.preprocessing import image
#from tensorflow.keras.preprocessing.image import ImageDataGenerator 

from ImageDataAugmentor.image_data_augmentor import *
import albumentations

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
# LearningRateScheduler
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.regularizers import l2
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
#from tensorflow.keras.applications import EfficientNetB6 
from tensorflow.keras.layers import *
import efficientnet.tfkeras as efn

from sklearn.model_selection import train_test_split, StratifiedKFold

import PIL
from PIL import ImageOps, ImageFilter
# increasing the default size of the charts
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
# graphs in svg look better
%config InlineBackend.figure_format = 'svg' 
%matplotlib inline

print('Python          :', sys.version.split('\n')[0])
print('Numpy           :', np.__version__)
print('Tensorflow      :', tf.__version__)
print('Keras           :', tf.keras.__version__)
print('Albumentations  :', albumentations.__version__)

In [24]:
# Checking that GPU is working
tf.test.gpu_device_name()

In [25]:
!pip freeze > requirements.txt

## Setup

In [26]:
# In setup, we make the basic settings: it's more convenient to sort them out in the future.

EPOCHS               = 5  # number of epochs for learning
BATCH_SIZE           = 64 # reduce the batch if the network is large, otherwise it will not fit into the memory on the GPU
LR                   = 1e-3
VAL_SPLIT            = 0.2 # the proportion of data allocated to the test = 20%

CLASS_NUM            = 10  # the number of classes in task
IMG_SIZE             = 224 # size of pictures that we send in model
IMG_CHANNELS         = 3   # RGB has 3 channel
input_shape          = (IMG_SIZE, IMG_SIZE, IMG_CHANNELS)

DATA_PATH = '../input/sf-dl-car-classification/' # data directory
PATH = "../working/car/" # working directory
PATH_BEST_MODEL = "../working/" # directory for save and load model when model is training

In [27]:
# Setting a specific random seed value for reproducibility
try:
    os.makedirs(PATH,exist_ok=True)
except FileExistsError:
    print("File exists: '../working/car/'")
    
try:
    os.makedirs(PATH_BEST_MODEL,exist_ok=True)
except FileExistsError:
    print("File exists: '../working/calc_models/'")

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)  
PYTHONHASHSEED = 0

## EDA

In [28]:
train_df = pd.read_csv(DATA_PATH+"train.csv")
sample_submission = pd.read_csv(DATA_PATH+"sample-submission.csv")
train_df.head()

In [29]:
train_df.info()

In [30]:
train_df.Category.value_counts()

### Preliminary conclusion:
In the task block when building the Baseline, it is specified: "The distribution of classes is fairly uniform — that's good." This does not seem to be true, since there is a difference of more than 30% in the amount of data for individual classes in the training sample. You can upload data from other sources for a more even distribution of classes.

In [31]:
print('Unzip pictures')
# Will unzip the files so that you can see them..
for data_zip in ['train.zip', 'test.zip']:
    with zipfile.ZipFile(DATA_PATH+data_zip,"r") as z:
        z.extractall(PATH)
        
print(os.listdir(PATH))

In [32]:
print('Picture examples (random sample)')
plt.figure(figsize=(12,8))

random_image = train_df.sample(n=9)
random_image_paths = random_image['Id'].values
random_image_cat = random_image['Category'].values

for index, path in enumerate(random_image_paths):
    im = PIL.Image.open(PATH+f'train/{random_image_cat[index]}/{path}')
    plt.subplot(3,3, index+1)
    plt.imshow(im)
    plt.title('Class: '+str(random_image_cat[index]))
    plt.axis('off')
plt.show()

In [33]:
# An example of a picture with dimensions to understand how best to process and compress images
image = PIL.Image.open(PATH + '/train/0/100380.jpg')
imgplot = plt.imshow(image)
plt.show()
image.size

## Data preparation

### Augmentation Data

In [36]:
# Creating a function for image data generator with augmentations transformations 
# I choose transformations that more approximate to images in task 
# that why I exclude transformations and distortions of forms of cars 
# but we can try use it in training model
def augment_data():
    AUGMENTATIONS = albumentations.Compose([
        albumentations.CLAHE(p=0.25, clip_limit=(1, 10), tile_grid_size=(10, 10)),
        albumentations.ChannelShuffle(p=0.25),
        #albumentations.Downscale(p=0.25, scale_min=0.75, scale_max=0.99, interpolation=1),
        #albumentations.ElasticTransform(p=0.25, alpha=1.0, sigma=10, alpha_affine=10, 
        #                                interpolation=1, border_mode=1, value=(0, 0, 0), 
        #                                mask_value=None, approximate=False),
        albumentations.Equalize(p=0.25, mode='cv', by_channels=True),
        albumentations.GaussNoise(p=0.25, var_limit=(10.0, 500.0), mean=-10),
        #albumentations.GridDistortion(p=0.25, num_steps=15, distort_limit=(-0.3, 0.3), 
        #                              interpolation=3, border_mode=1, value=(0, 0, 0), 
        #                              mask_value=None),
        albumentations.HorizontalFlip(p=0.5),
        albumentations.HueSaturationValue(p=0.5, hue_shift_limit=(-20, 20), 
                                          sat_shift_limit=(-20, 20), val_shift_limit=(-20, 20)),
        #albumentations.ISONoise(p=0.5, intensity=(0.1, 0.4), color_shift=(0.01, 0.3)),
        albumentations.MotionBlur(p=0.25, blur_limit=(3, 7)),
        #albumentations.OpticalDistortion(p=0.25, distort_limit=(-0.3, 0.3), 
        #                                 shift_limit=(-0.2, 0.2), interpolation=2, 
        #                                 border_mode=1, value=(0, 0, 0), mask_value=None),
        albumentations.RGBShift(p=0.5),
        albumentations.OneOf([
            albumentations.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2),
            albumentations.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1),
            ],p=0.5),
        albumentations.RandomGamma(p=0.25, gamma_limit=(100, 130), eps=1e-07),
        #albumentations.RandomSnow(p=0.25, snow_point_lower=0.25, snow_point_upper=0.75, 
        #                          brightness_coeff=1.2),
        albumentations.Rotate(p=0.5, limit=(-5, 5), interpolation=2, border_mode=2)
    ])

    train_datagen = ImageDataAugmentor(
        rescale=1./255,
        validation_split=VAL_SPLIT,
        seed=RANDOM_SEED,
        augment=AUGMENTATIONS,
        preprocess_input=None)

    test_datagen = ImageDataAugmentor(rescale=1./255,
                                      seed=RANDOM_SEED,
                                      )
    return train_datagen, test_datagen

train_datagen, test_datagen = augment_data()

### Generation Data

In [37]:
# Create data generators
def generators(IMG_SIZE=IMG_SIZE, BATCH_SIZE_FOR_GEN=BATCH_SIZE):
    train_generator = train_datagen.flow_from_directory(
        PATH+'train/',      # the directory where the folders with pictures are located
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE_FOR_GEN,
        class_mode='categorical',
        shuffle=True, 
        subset='training') # set as training data

    test_generator = train_datagen.flow_from_directory(
        PATH+'train/',
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE_FOR_GEN,
        class_mode='categorical',
        shuffle=True, 
        subset='validation') # set as validation data

    test_sub_generator = test_datagen.flow_from_dataframe( 
        dataframe=sample_submission,
        directory=PATH+'test_upload/',
        x_col="Id",
        y_col=None,
        shuffle=False,
        class_mode=None,
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE_FOR_GEN,)
    return train_generator, test_generator, test_sub_generator

train_generator, test_generator, test_sub_generator = generators()

In [38]:
# Some examples of pictures in train_generator
def imshow(image_RGB):
    io.imshow(image_RGB)
    io.show()

x,y = train_generator.next()
print('Some examples of pictures from train_generator')
plt.figure(figsize=(12,8))

for i in range(0,6):
    image = x[i]
    plt.subplot(3,3, i+1)
    plt.imshow(image)
    #plt.title('Class: '+str(y[i]))
    #plt.axis('off')
plt.show()

In [39]:
x,y = test_generator.next()
print('Some examples of pictures from test_generator')
plt.figure(figsize=(12,8))

for i in range(0,6):
    image = x[i]
    plt.subplot(3,3, i+1)
    plt.imshow(image)
    #plt.title('Class: '+str(y[i]))
    #plt.axis('off')
plt.show()

## Functions in calculations

In [40]:
# Callbacks that used for training model
def callbacks(lr):
    checkpoint = ModelCheckpoint(PATH_BEST_MODEL+'best_model.hdf5', monitor='val_accuracy', 
                                 verbose=1, mode='max', save_best_only=True)
    earlystop = EarlyStopping(monitor='val_accuracy', patience=6, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.2, patience=3, verbose=1,
                                  min_lr=lr/100)
    return [checkpoint, earlystop, reduce_lr]

# patience=3 on ReduceLROnPlateau in Steps: 1,2,3,4,5_1,5_2, 
# patience=2 on ReduceLROnPlateau in Steps: 5_3,5_4, 

In [41]:
# Function to save pictures of training model history
def plot_save_fig(history, num_step,
                  PATH_BEST_MODEL=PATH_BEST_MODEL):
    
    """
    history - create from model.fit
    num_step - integer which is a sequential number step in fine-tunning model
    
    """
    
    plt.figure(figsize=(10,5))
    #plt.style.use('dark_background')
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(len(acc))

    plt.plot(epochs, acc, 'b', label='Training acc')
    plt.plot(epochs, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.savefig(PATH_BEST_MODEL + 'Train_Vall_acc_st_' + str(num_step) + '.png')

    #plt.figure()
    plt.figure(figsize=(10,5))
    #plt.style.use('dark_background')
    plt.plot(epochs, loss, 'b', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()
    plt.savefig(PATH_BEST_MODEL + 'Train_Vall_loss_st_' + str(num_step) + '.png')

In [42]:
# Function to print pictures of training model history in notebook without locally saving
def plot_history(history):
    
    """
    history - create from model.fit
        
    """
    
    plt.figure(figsize=(10,5))
    #plt.style.use('dark_background')
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(len(acc))

    plt.plot(epochs, acc, 'b', label='Training acc')
    plt.plot(epochs, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()

    #plt.figure()
    plt.figure(figsize=(10,5))
    #plt.style.use('dark_background')
    plt.plot(epochs, loss, 'b', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

    plt.show()

In [43]:
# Function to create model to load earlier saved best weights from training model

def build_best_model(f_tune_coef, LR, 
                     train_all_base_layers=False,
                     PATH=PATH_BEST_MODEL,
                     WEIGHTS_NAME='best_model.hdf5',
                     input_shape=input_shape, 
                     BATCH_SIZE=BATCH_SIZE, 
                     trainable=True):
    
    """
    f_tune_coef - coefficient that define integer number "fine_tune_at" 
                  layers before fine_tune_at - will be freeze
                  layers onwards - will be Fine-tuned (works with train_all_base_layers==False),
    train_all_base_layers - if True all layers will be trainable (f_tune_coef - any from 1 to 588),
    LR - learning rate,
    PATH - directory with files of previous saved weights of model to be loaded
    WEIGHTS_NAME - name of file of previous saved weights of model to be loaded
    input_shape, BATCH_SIZE - size of pictures & batch size may be changed in steps of training model
    
    """
    
  
    base_model = efn.EfficientNetB6(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = trainable
    
    # Сhoose layers which weights will train and freeze  
    if train_all_base_layers==False:
        # Fine-tune from this layer onwards
        fine_tune_at = int(len(base_model.layers)//f_tune_coef)

        # Freeze all the layers before the `fine_tune_at` layer
        for layer in base_model.layers[:fine_tune_at]:
            layer.trainable =  False

    # Creation new head:
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.25)(x)
    
    #  logistic layer -- 10 classes
    predictions = Dense(CLASS_NUM, activation='softmax')(x)

    #  this is the model we will train
    model = Model(inputs=base_model.input, outputs=predictions)

    print("Number of trainable_variables in model: ", 
          len(model.trainable_variables))

    model.compile(loss="categorical_crossentropy", 
                  optimizer=optimizers.Adam(learning_rate=LR), 
                  metrics=["accuracy"])

    model.load_weights(PATH + WEIGHTS_NAME)
    
    return model

## Step 1

In [ ]:
input_shape

In [ ]:
base_model = efn.EfficientNetB6(weights='imagenet', include_top=False, input_shape=input_shape)
#print(base_model.summary() # to see summary information about base_model

# first: train only the top layers (which were randomly initialized)
base_model.trainable = False

# Creation new head:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

#  logistic layer -- 10 classes
predictions = Dense(CLASS_NUM, activation='softmax')(x)

#  this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# numbers of layers and training variables
print(len(model.layers))
print(len(model.trainable_variables))

### Training

In [ ]:
LR=0.0001
model.compile(loss="categorical_crossentropy", 
              optimizer=optimizers.Adam(learning_rate=LR), 
              metrics=["accuracy"])

scores = model.evaluate(test_generator, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

callbacks_list = callbacks(lr=LR)

In [ ]:
# Training
history = model.fit(
        train_generator,
        steps_per_epoch = train_generator.samples//train_generator.batch_size, #len(train_generator), #
        validation_data = test_generator,
        validation_steps = test_generator.samples//test_generator.batch_size, #len(test_generator), #
        epochs = EPOCHS,
        callbacks = callbacks_list
)

plot_history(history)

In [ ]:
# Save model & load best iteration after fitting (best_model):
model.save(PATH_BEST_MODEL + 'model_step1.hdf5')

# to save best model weights in file with individual name
model.load_weights(PATH_BEST_MODEL + 'best_model.hdf5') 
model.save_weights(PATH_BEST_MODEL + 'best_model_st_1.hdf5')

scores = model.evaluate(test_generator, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

## Step 2

In [ ]:
LR=0.00005
BATCH_SIZE=32

# if model continues training by other session
# download file with weights 'best_model_st_1.hdf5' to PATH='../input/weights-aft-st-1/'
model = build_best_model(f_tune_coef=2, LR=LR,
                         #PATH='../input/weights-aft-st-1/',
                         #WEIGHTS_NAME='best_model_st_1.hdf5'
                        )

train_datagen, test_datagen = augment_data()
train_generator, test_generator, sub_test_generator = generators(BATCH_SIZE_FOR_GEN=BATCH_SIZE)
callbacks_list = callbacks(lr=LR)

In [ ]:
# Training
history = model.fit(
        train_generator,
        steps_per_epoch = train_generator.samples//train_generator.batch_size,
        validation_data = test_generator, 
        validation_steps = test_generator.samples//test_generator.batch_size,
        epochs = 10,
        callbacks = callbacks_list
)

plot_history(history)

In [ ]:
model.save(PATH_BEST_MODEL + 'model_step2.hdf5')

# to save best model weights in file with individual name
model.load_weights(PATH_BEST_MODEL + 'best_model.hdf5') 
model.save_weights(PATH_BEST_MODEL + 'best_model_st_2.hdf5')

scores = model.evaluate(test_generator, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

## Step 3

In [ ]:
LR=0.00005
BATCH_SIZE=32

# if model continues training by other session
# download file with weights 'best_model_st_2.hdf5' to PATH='../input/weights-aft-st-2/'
model = build_best_model(f_tune_coef=4, LR=LR,
                         #PATH='../input/weights-aft-st-2/',
                         #WEIGHTS_NAME='best_model_st_2.hdf5'
                        )

train_datagen, test_datagen = augment_data()
train_generator, test_generator, sub_test_generator = generators(BATCH_SIZE_FOR_GEN=BATCH_SIZE)
callbacks_list = callbacks(lr=LR)

In [ ]:
# Training
history = model.fit(
        train_generator,
        steps_per_epoch = train_generator.samples//train_generator.batch_size,
        validation_data = test_generator, 
        validation_steps = test_generator.samples//test_generator.batch_size,
        epochs = 10,
        callbacks = callbacks_list
)

plot_history(history)

In [ ]:
model.save(PATH_BEST_MODEL + 'model_step3.hdf5')

# to save best model weights in file with individual name
model.load_weights(PATH_BEST_MODEL + 'best_model.hdf5') 
model.save_weights(PATH_BEST_MODEL + 'best_model_st_3.hdf5')

scores = model.evaluate(test_generator, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

## Step 4

In [ ]:
LR=0.00001
BATCH_SIZE=16

# if model continues training by other session
# download file with weights 'best_model_st_3.hdf5' to PATH='../input/weights-aft-st-3/'
model = build_best_model(f_tune_coef=588, LR=LR,
                         train_all_base_layers=True,
                         #PATH='../input/weights-aft-st_3/',
                         #WEIGHTS_NAME='best_model_st_3.hdf5'
                        )

train_datagen, test_datagen = augment_data()
train_generator, test_generator, sub_test_generator = generators(BATCH_SIZE_FOR_GEN=BATCH_SIZE)
callbacks_list = callbacks(lr=LR)

# Check the trainable status of the individual layers
#for layer in model.layers:
#    print(layer, layer.trainable) 

In [ ]:
# Training
history = model.fit(
        train_generator,
        steps_per_epoch = train_generator.samples//train_generator.batch_size,
        validation_data = test_generator, 
        validation_steps = test_generator.samples//test_generator.batch_size,
        epochs = 10,
        callbacks = callbacks_list
)

plot_history(history)

In [ ]:
model.save(PATH_BEST_MODEL + 'model_step4.hdf5')

# to save best model weights in file with individual name
model.load_weights(PATH_BEST_MODEL + 'best_model.hdf5')
model.save_weights(PATH_BEST_MODEL + 'best_model_st_4.hdf5')

scores = model.evaluate(test_generator, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

## Step 5_1 + Step 5_2

In [ ]:
EPOCHS               = 10
BATCH_SIZE           = 4 
LR                   = 1e-5
VAL_SPLIT            = 0.2

CLASS_NUM            = 10
IMG_SIZE             = 448
IMG_CHANNELS         = 3
input_shape          = (IMG_SIZE, IMG_SIZE, IMG_CHANNELS)

In [ ]:
# if model continues training by other session
# download file with weights 'best_model_st_4.hdf5' to PATH='../input/weights-aft-st-4/'
model = build_best_model(f_tune_coef=588, LR=LR,
                         train_all_base_layers=True, 
                         #PATH='../input/weights-aft-st-4/',
                         #WEIGHTS_NAME='best_model_st_4.hdf5'
                        )

In [ ]:
# Creation new data generators due to IMG_SIZE increase and BATCH_SIZE changes

AUGMENTATIONS = albumentations.Compose([
    albumentations.CLAHE(p=0.25, clip_limit=(1, 10), tile_grid_size=(10, 10)),
    albumentations.ChannelShuffle(p=0.25),
    albumentations.Equalize(p=0.25, mode='cv', by_channels=True),
    albumentations.HorizontalFlip(p=0.5),
    albumentations.MotionBlur(p=0.25, blur_limit=(3, 7)),
    albumentations.RGBShift(p=0.5),
    albumentations.OneOf([
        albumentations.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2),
        albumentations.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1),
    ],p=0.5),
    albumentations.Rotate(p=0.5, limit=(-5, 5), interpolation=2, border_mode=2)
])

train_datagen = ImageDataAugmentor(
    rescale=1./255,
    validation_split=VAL_SPLIT,
    seed=RANDOM_SEED,
    augment=AUGMENTATIONS,
    preprocess_input=None)

train_generator = train_datagen.flow_from_directory(
    PATH+'train/',     # the directory where the folders with pictures are located
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, 
    subset='training') # set as training data

test_generator = train_datagen.flow_from_directory(
    PATH+'train/',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, 
    subset='validation') # set as validation data

callbacks_list = callbacks(lr=LR)

In [ ]:
# Training
history = model.fit(
        train_generator,
        steps_per_epoch = train_generator.samples//train_generator.batch_size, 
        validation_data = test_generator,
        validation_steps = test_generator.samples//test_generator.batch_size, 
        epochs = 8,
        callbacks = callbacks_list
)

plot_history(history)

In [ ]:
model.save(PATH_BEST_MODEL + 'model_step5_1.hdf5')

# to save best model weights in file with individual name
model.load_weights(PATH_BEST_MODEL + 'best_model.hdf5')
model.save_weights(PATH_BEST_MODEL + 'best_model_st_5_1.hdf5')

scores = model.evaluate(test_generator, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

### 'best_model_st_5_2.hdf5' - continues to fit the model (+6 epoch) with the same params after 'best_model_st_5_1.hdf5' (8 epoch)

In [ ]:
# Training
history = model.fit(
        train_generator,
        steps_per_epoch = train_generator.samples//train_generator.batch_size, 
        validation_data = test_generator,
        validation_steps = test_generator.samples//test_generator.batch_size, 
        epochs = 6,
        callbacks = callbacks_list
)

plot_history(history)

In [ ]:
model.save(PATH_BEST_MODEL + 'model_step5_2.hdf5')

# to save best model weights in file with individual name
model.load_weights(PATH_BEST_MODEL + 'best_model.hdf5')
model.save_weights(PATH_BEST_MODEL + 'best_model_st_5_2.hdf5')

scores = model.evaluate(test_generator, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

## Step 5_3

In [44]:
EPOCHS               = 8
BATCH_SIZE           = 4 
LR                   = 1e-6
VAL_SPLIT            = 0.2

CLASS_NUM            = 10
IMG_SIZE             = 448
IMG_CHANNELS         = 3
input_shape          = (IMG_SIZE, IMG_SIZE, IMG_CHANNELS)

In [ ]:
# if model continues training by other session
# download file with weights 'best_model_st_5_2.hdf5' to PATH='../input/weights-aft-st-5_2/'
model = build_best_model(f_tune_coef=588, LR=LR,
                         train_all_base_layers=True,
                         #PATH='../input/weights-aft-st-5-2/',
                         #WEIGHTS_NAME='best_model_st_5_2.hdf5'
                        )

In [ ]:
AUGMENTATIONS = albumentations.Compose([
    albumentations.CLAHE(p=0.25, clip_limit=(1, 10), tile_grid_size=(10, 10)),
    albumentations.ChannelShuffle(p=0.25),
    albumentations.Equalize(p=0.25, mode='cv', by_channels=True),
    albumentations.HorizontalFlip(p=0.5),
    albumentations.MotionBlur(p=0.25, blur_limit=(3, 7)),
    albumentations.RGBShift(p=0.5),
    albumentations.OneOf([
        albumentations.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2),
        albumentations.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1),
    ],p=0.5),
    albumentations.Rotate(p=0.5, limit=(-5, 5), interpolation=2, border_mode=2)
])

train_datagen = ImageDataAugmentor(
    rescale=1./255,
    validation_split=VAL_SPLIT,
    seed=RANDOM_SEED,
    augment=AUGMENTATIONS,
    preprocess_input=None)

train_generator = train_datagen.flow_from_directory(
    PATH+'train/',     # the directory where the folders with pictures are located 
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, 
    subset='training') # set as training data

test_generator = train_datagen.flow_from_directory(
    PATH+'train/',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, 
    subset='validation') # set as validation data

# change patience form 3 to 2 on reduceOnPlateou
callbacks_list = callbacks(lr=LR)

In [ ]:
# Training
history = model.fit(
        train_generator,
        steps_per_epoch = train_generator.samples//train_generator.batch_size, 
        validation_data = test_generator,
        validation_steps = test_generator.samples//test_generator.batch_size, 
        epochs = EPOCHS,
        callbacks = callbacks_list
)

plot_history(history)

In [ ]:
model.save(PATH_BEST_MODEL + 'model_step5_3.hdf5')

# to save best model weights in file with individual name
model.load_weights(PATH_BEST_MODEL + 'best_model.hdf5')
model.save_weights(PATH_BEST_MODEL + 'best_model_st_5_3.hdf5')

scores = model.evaluate(test_generator, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

## TTA

In [45]:
def save_submit_tta(PATH_TO_MODEL=PATH_BEST_MODEL, 
                    WEIGHTS_NAME='best_model.hdf5',
                    RANDOM_SEED=42,
                    steps_for_tta=10):
    
    AUGMENTATIONS = albumentations.Compose([
        albumentations.CLAHE(p=0.25, clip_limit=(1, 10), tile_grid_size=(10, 10)),
        albumentations.ChannelShuffle(p=0.25),
        albumentations.Equalize(p=0.25, mode='cv', by_channels=True),
        albumentations.HorizontalFlip(p=0.5),
        #albumentations.MotionBlur(p=0.25, blur_limit=(3, 7)),
        #albumentations.RGBShift(p=0.5),
        #albumentations.OneOf([
        #    albumentations.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2),
        #    albumentations.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1),
        #],p=0.5),
        albumentations.Rotate(p=0.5, limit=(-5, 5), interpolation=2, border_mode=2)
    ])

    test_sub_datagen = ImageDataAugmentor(
        rescale=1./255,
        validation_split=VAL_SPLIT,
        seed=RANDOM_SEED,
        augment=AUGMENTATIONS,
        preprocess_input=None)

    test_sub_generator = test_sub_datagen.flow_from_dataframe( 
        dataframe=sample_submission,
        directory=PATH+'test_upload/',
        x_col="Id",
        y_col=None,
        shuffle=False,
        class_mode=None,
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE)
    
    model = build_best_model(f_tune_coef=588, LR=LR,
                             PATH=PATH_TO_MODEL,
                             WEIGHTS_NAME=WEIGHTS_NAME,
                             train_all_base_layers=True)
    
    tta_steps = steps_for_tta
    predictions_list = []
    
    for i in range(tta_steps):
        test_sub_generator.reset()
        preds = model.predict(test_sub_generator, steps=len(test_sub_generator),  verbose=1) 
        predictions_list.append(preds)
    pred = np.mean(predictions_list, axis=0)
    
    predictions = np.argmax(pred, axis=-1) #multiple categories
    label_map = (train_generator.class_indices)
    label_map = dict((v,k) for k,v in label_map.items()) #flip k,v
    predictions = [label_map[k] for k in predictions]
    
    filenames_with_dir=test_sub_generator.filenames
    submission = pd.DataFrame({'Id':filenames_with_dir, 'Category':predictions}, columns=['Id', 'Category'])
    submission['Id'] = submission['Id'].replace('test_upload/','')
    submission.to_csv('submission_st_5_4.csv', index=False)
    print('Save submit')
    return submission

In [46]:
submit_DF = save_submit_tta(PATH_TO_MODEL='../input/weights-aft-st-5-3/',
                            WEIGHTS_NAME='best_model_st_5_3.hdf5')

In [ ]:
# Clean PATH
#import shutil
#shutil.rmtree(PATH)

**Что еще можно сделать для улучшения модели!**


→ Попробовать другие архитектуры сетей, например, SOTA на ImageNet позднее B6, дающие бОльшую точность.

→ Поэкспериментировать с архитектурой «головы».

→ Попробовать больше эпох на 5 этапе обучения.

→ Использовать разные техники управления Learning Rate, например Sheduler, но он требует большего времени и исключение ограничения на сессию.

→ Использовать внешние датасеты для дообучения модели.

→ Обернуть модель в сервис на Flask (чтобы на практике отследить особенности внедрения DL-моделей в продакшн).